In [1]:
import requests
import urllib.parse
from bs4 import BeautifulSoup

In [2]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://www.ejawi.net/result.php"

headers = CaseInsensitiveDict()
headers["authority"] = "www.ejawi.net"
headers["accept"] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
headers["accept-language"] = "en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7"
headers["cache-control"] = "no-cache"
headers["content-type"] = "application/x-www-form-urlencoded"
headers["cookie"] = "__utma=248434904.1854521706.1653239537.1653239537.1653239537.1; __utmc=248434904; __utmz=248434904.1653239537.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; __utmb=248434904.1.10.1653239537"
headers["origin"] = "https://www.ejawi.net"
headers["pragma"] = "no-cache"
headers["referer"] = "https://www.ejawi.net/converterV2.php?go=rumi"
headers["sec-ch-ua"] = '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"'
headers["sec-ch-ua-mobile"] = "?0"
headers["sec-ch-ua-platform"] = '"macOS"'
headers["sec-fetch-dest"] = "iframe"
headers["sec-fetch-mode"] = "navigate"
headers["sec-fetch-site"] = "same-origin"
headers["sec-fetch-user"] = "?1"
headers["upgrade-insecure-requests"] = "1"
headers["user-agent"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"

In [3]:
def get_jawi(string):
    data = f"phonText={string}&disable_direction=on&number=1"
    resp = requests.post(url, headers=headers, data=data)
    decoded = resp._content.decode()
    soup = BeautifulSoup(decoded, 'html.parser')
    return soup.find_all('td')[0].text

In [4]:
get_jawi('"hello"')

'"هيللو"'

In [8]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/pure-text/dumping-news.txt

In [9]:
!head -n 10 dumping-news.txt

Lawatan Putera Mahkota Arab Saudi bincang pelaburan, ekonomi
KUALA LUMPUR: Lawatan Putera Mahkota Arab Saudi, Mohammed bin Salman ke Malaysia pada Ahad ini terarah kepada pertemuan bagi membincangkan isu ekonomi dan pelaburan antara kedua-dua negara.
Menteri Luar, Datuk Saifuddin Abdullah, berkata tujuan lawatan kerabat diraja Arab Saudi itu akan diperjelaskan oleh Kementerian Hal Ehwal Ekonomi.
"Saya rasa elok perkara itu (lawatan Putera Mahkota Arab Saudi) dijelaskan sendiri oleh (Menteri Hal Ehwal Ekonomi) Datuk Seri Mohamed Azmin (Ali).
"Selain saya (keluarkan kenyataan mengenai lawatan itu), kamu (pemberita) juga boleh ajukan soalan kepada Datuk Seri Mohamed Azmin.
"Ia banyak berkisar ekonomi dan perniagaan," katanya kepada pemberita selepas hadir di majlis meja bulat bersama belia di Institut Diplomasi dan Hubungan Luar Negeri, di sini, hari ini.
Lawatan Putera Mahkota Arab Saudi ke Malaysia akan berlangsung selama dua hari berakhir Isnin ini.

Isu halal dominasi lawatan Mujahid 

In [10]:
import json
import random

In [11]:
with open('dumping-news.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
len(data)

1791784

In [12]:
def window(string, n = 4):
    splitted = string.split()
    return [' '.join(splitted[i: i + n]) for i in range(0, len(splitted), n)]

window(data[0])

['Lawatan Putera Mahkota Arab', 'Saudi bincang pelaburan, ekonomi']

In [21]:
import mp
from tqdm import tqdm

In [22]:
windows = []
for i in tqdm(range(len(data))):
    windows.extend(window(data[i], n = random.randint(10, 20)))

100%|██████████████████████████████████████████████████████████████████████████████████| 1791784/1791784 [00:03<00:00, 496294.40it/s]


In [23]:
len(windows)

3332675

In [24]:
import re
from unidecode import unidecode

_punctuation = "!'(),.:;? +"
_special = '-'
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
_numbers = '0123456789'
_small_letters = 'abcdefghijklmnopqrstuvwxyz'

SYMBOLS = (
    list(_special) + list(_punctuation) + list(_letters) + list(_numbers)
)

def cleaning(string):
    string = unidecode(string)
    string = ''.join([c if c in SYMBOLS else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    return string

In [25]:
from tqdm import tqdm

def loop(files):
    windows, _ = files
    results = []
    for d in tqdm(windows):
        try:
            t = cleaning(d)
            if len(t):
                r = get_jawi(t)
                if len(r):
                    results.append((t, r))
        except Exception as e:
            print(e)
    return results
        

In [27]:
r = loop((windows[:10],0))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.39it/s]


In [29]:
r = mp.multiprocessing(windows, loop, cores = 20)

 11%|█████████▎                                                                          | 18398/166633 [1:09:21<12:46:35,  3.22it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 12%|██████████                                                                           | 19620/166633 [1:13:31<7:21:32,  5.55it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 12%|██████████▎                                                                    

In [30]:
with open('wikipedia-news-v2.json', 'w') as fopen:
    json.dump(r, fopen)

In [31]:
len(r)

3332675

In [36]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(r, test_size = 100000)

In [37]:
test[-10:]

[('diluluskan, kita angkat (bawa) kepada seri paduka baginda yang di-pertuan agong untuk mendapat perkenan tuanku',
  'دلولوسكن، كيت اڠكت ﴿باوا﴾ كڤد سري ڤدوك بڬيندا يڠ د-ڤرتوان اڬوڠ اونتوق منداڤت ڤركنن توانكو'),
 ('remaja perempuan berusia 14 tahun yang dilaporkan hilang 25 mac lalu di miri dipercayai menjadi mangsa rogol secara bergilir-gilir oleh',
  'رماج ڤرمڤوان بروسيا ١٤ تاهون يڠ دلاڤوركن هيلڠ ٢٥ مچ لالو د ميري دڤرچايأي منجادي مڠسا روڬول سچارا برڬيلير-ڬيلير اوليه'),
 ('kuala kangsar: seorang lelaki orang kurang upaya (oku) lemas ketika mandi di kawasan air terjun batu hampar di',
  'كوالا كڠسر: سأورڠ للاكي اورڠ كورڠ اوڤاي ﴿اوكو﴾ لمس كتيك مندي د كاوسن اير ترجون باتو همڤر د'),
 ('jumlah pemegang unit bagi simpanan rm500,000 dan ke atas pula seramai 23,304 atau mewakili 0.2 peratus dengan purata simpanan yang',
  'جومله ڤمڬڠ اونيت باڬي سيمڤنن رم٥٠٠،٠٠٠ دان ك اتس ڤولا سراما ٢٣،٣٠٤ اتاو مواكيلي ٠.٢ ڤراتوس دڠن ڤوراتا سيمڤنن يڠ'),
 ('rakyat berbilang kaum dan agama di negara ini sentiasa

In [38]:
with open('jawi-rumi-news-full.train', 'w') as fopen:
    for i in range(len(train)):
        fopen.write(json.dumps(train[i]) + '\n')

In [39]:
with open('jawi-rumi-news-full.test', 'w') as fopen:
    for i in range(len(test)):
        fopen.write(json.dumps(train[i]) + '\n')

In [40]:
t = []
with open('jawi-rumi-news-full.test') as fopen:
    for r in fopen:
        t.append(json.loads(r))